# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

# Importing dataset

In [112]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned.csv')

# QUESTION 4:

# 1. MALES OR FEMALES COMMIT SUICIDE MORE?

In [113]:
suicides.dtypes

Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year              int64
GDP/Capita            int64
Generation           object
dtype: object

In [114]:
suicides = suicides.astype({"Year": 'object'})

In [115]:
# From 1990 till 2015

In [116]:
tot_s_mf = suicides[['Gender','Num_Suicides','Year']].groupby('Gender').sum().reset_index()

In [117]:
tot_s_mf['Percentage'] = tot_s_mf['Num_Suicides'].apply(lambda row: (row/sum(tot_s_mf['Num_Suicides'])*100))

In [118]:
tot_s_mf = tot_s_mf.round({'Percentage': 2})

## 1. Solution: 
## TOTAL SUICIDES MALES AND FEMALES 1990 - 2015:
## males  77.3%, females: 22.7%

In [120]:
tot_s_mf

,Gender,Num_Suicides,Percentage
0,female,1380293,22.67
1,male,4707679,77.33


# 2. Males or females commit suicide more in 2015?

In [121]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [170]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum().reset_index()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [171]:
s_gender2015

,Gender,Num_Suicides
0,female,47248
1,male,156392


In [172]:
s_gender2015['Percentage'] = s_gender2015['Num_Suicides'].apply(lambda row: (row/sum(s_gender2015['Num_Suicides'])*100))

In [174]:
s_gender2015

,Gender,Num_Suicides,Percentage
0,female,47248,23.201729
1,male,156392,76.798271


In [175]:
s_gender2015 = s_gender2015.round({'Percentage': 2})

# 2. SOLUTION PECENTAGE IN 2015: males 76.8%, females 23.2%

In [176]:
s_gender2015

,Gender,Num_Suicides,Percentage
0,female,47248,23.2
1,male,156392,76.8


# 3. TOP 10 COUNTRIES BY GENRE:

## BY NUMBER:

### Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [177]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [178]:
top10countriesbynum

,Num_Suicides
Country,
United States,44189
Russian Federation,25432
Japan,23092
Republic of Korea,13510
Brazil,11163
Germany,10088
Ukraine,7574
Mexico,6234
Poland,5420


### MALES : TOP 10 COUNTRIES BY NUM

In [179]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [180]:
top10malesbynum

,Num_Suicides
Country,
United States,33990
Russian Federation,20861
Japan,16146
Republic of Korea,9556
Brazil,8766
Germany,7403
Ukraine,6148
Mexico,4986
Poland,4697


### FEMALES : TOP 10 COUNTRIES BY NUM

In [181]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [182]:
top10femalesbynum

,Num_Suicides
Country,
United States,10199
Japan,6946
Russian Federation,4571
Republic of Korea,3954
Germany,2685
Brazil,2397
Ukraine,1426
Mexico,1248
United Kingdom,1178


In [183]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10femalesbynum.index)
idgnm = pd.Index(top10malesbynum.index)

In [184]:
set_intersection = idgnf.intersection(idgnm)

In [185]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### As we can see 9 out of 10 countries match

In [190]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

Index(['Spain'], dtype='object', name='Country')

In [191]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

Index(['Poland'], dtype='object', name='Country')

In [192]:
set_intersection = idgnf.intersection(idgnm)

In [193]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

## Now we realize that we ook into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:


## BY SIZE:

In [201]:
suicides_rateallyears = suicides[['Country', 'Population', 'Num_Suicides']].groupby('Country').sum()

In [202]:
suicides_rateallyears.head()

,Population,Num_Suicides
Country,,
Albania,54048467,1766
Antigua and Barbuda,1687321,11
Argentina,897040831,71060
Armenia,74948140,1838
Aruba,1259677,101


In [203]:
suicides_country_2015 = suicides[suicides["Year"] == 2015].groupby("Country").sum()
suicides_country_2015

,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita
Country,,,,,
Antigua and Barbuda,1,91889,15.62,16378356444,178236
Argentina,3073,39699624,112.13,7136991424956,179772
Armenia,74,2795335,45.28,126640052076,45300
Australia,3027,22240785,154.18,16188408353436,727872
Austria,1251,8219386,194.62,4584791163696,557808
Belgium,1867,10624356,196.66,5460474757596,513960
Belize,26,319835,135.46,21342811200,66732
Brazil,11163,191087085,77.45,21626572484892,113172
Chile,1838,16565725,147.68,2927993732856,176748


In [200]:
rate2015.head()

,Country,Population,Num_Suicides,Suicides/Population
0,Albania,54048467,1766,3.267438
1,Antigua and Barbuda,1687321,11,0.651921
2,Argentina,897040831,71060,7.921602
3,Armenia,74948140,1838,2.452362
4,Aruba,1259677,101,8.017928


In [196]:
suicides_rate['Rates'] = (suicides_rate['Num_Suicides']/suicides_rate['Population'])*100000

In [197]:
suicides_rate.head()

,Population,Num_Suicides,Rates
Country,,,
Albania,54048467,1766,3.267438
Antigua and Barbuda,1687321,11,0.651921
Argentina,897040831,71060,7.921602
Armenia,74948140,1838,2.452362
Aruba,1259677,101,8.017928


In [163]:
suicides_rate = suicides_rate.reset_index()

In [164]:
suicides_rate = suicides_rate[['Country','Rates']].groupby('Country').sum().sort_values("Rates", ascending=False)


In [165]:
top10size = suicides_rate.groupby('Country').sum().sort_values("Rates", ascending=False)[0:10]

In [166]:
top10size

,Rates
Country,
Lithuania,41.464101
Russian Federation,32.959289
Belarus,30.344685
Hungary,30.022407
Sri Lanka,28.872874
Latvia,28.471011
Kazakhstan,26.898614
Slovenia,26.360477
Estonia,25.964525


In [167]:
suicides_rate = suicides_rate.reset_index()

In [168]:
# Creating an intersection by index to see which countries appears in both rankings.
idx1 = pd.Index(top10size.index)
idx2 = pd.Index(top10countriesbynum.index)

In [169]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### MALE: TOP 10 COUNTRIES BY SIZE

In [68]:
top10malesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]



In [69]:
top10malesbysize

,Suicides/100kPop
Country,
Lithuania,323.31
Republic of Korea,285.19
Uruguay,223.03
Slovenia,217.67
Latvia,211.07
Ukraine,206.64
Hungary,203.39
Russian Federation,199.26
Kazakhstan,195.35


### FEMALE: TOP 10 COUNTRIES BY SIZE

In [ ]:
top10femalesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [ ]:
top10femalesbysize

In [ ]:
# Creating an intersection by index to see if the countries with a higher number of suicides 
# by size of the population are the same countries for both genres.

idgsf = pd.Index(top10femalesbysize.index)
idgsm = pd.Index(top10malesbysize.index)

In [ ]:
set_intersection = idgsf.intersection(idgsm)

In [ ]:
set_intersection

In [ ]:
# females but no males:

idgsf.difference(idgsm)

In [ ]:
# males but not females:

idgsm.difference(idgsf)

### In this case, the countries Republic of Korea,  Lithuania, Hungary, Croatia, Slovenia and Uruguay match for both genres.

### But Belgiun, Iceland, Japan and Serbia appear in the top 10 for females while Kazakhstan, Latvia, Russian Federation and Ukraine appear in the top 10 for males.

# QUESTION 5: 

# 1.  Which generation has committed suicide the most/least?


## First, we will see which generation committed more suicides from 1990 till 2015:

## BY NUMBER:

In [ ]:
df_1 = suicides[['Generation','Num_Suicides']].groupby('Generation').sum().sort_values('Num_Suicides', ascending = False)

In [ ]:
df_1

### We see that in number of suicides, 'Boomers Generation' is the generation than committed the highest number of suicides. What will happen if we divide the number of suicides per generation by the total population for each generation? Let's check:

In [ ]:
df_2 = suicides[['Generation','Num_Suicides', 'Suicides/100kPop']].groupby('Generation').sum().sort_values('Suicides/100kPop', ascending = False)

In [ ]:
df_2 = df_2.round({'Suicides/100kPop': 2})

In [ ]:
df_2

# in 2015:

In [ ]:
# checking the data 

suicides_2015.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Let's see which generation committes more suicides in number of suicides:

In [ ]:
suicides_15 = suicides_2015[['Generation','Num_Suicides', 'Population']]

In [ ]:
suicides_15.groupby(['Generation']).sum().sort_values('Num_Suicides', ascending = False)

### We see that Silent and Z Generations commit the less suicides. This make sense because Silent Generation is the oldest one and generation Z is the youngest one so, probably, the population is lower than other range of ages. Let's calculate Suicide per generation divided by the total population:

In [ ]:
suicides15 =suicides_15.groupby(['Generation']).sum()

In [ ]:
suicides15['Sui/Pop'] = (suicides15['Num_Suicides']/suicides15['Population'])*1000000

In [ ]:
suicides15.sort_values('Sui/Pop', ascending = False)

### In terms of population, we see that Silent Generation committes more suicides followed by the Boomer generation.

# QUESTION 6 Suicide/Population rate per country?

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES FROM 1990-2015:

In [ ]:
s_ratecountry = suicides[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [ ]:
s_ratecountry['Rate'] = (s_ratecountry['Num_Suicides']/s_ratecountry['Population'])*1000000

In [ ]:
s_ratecountry.sort_values('Rate', ascending = False)[0:10]

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES IN 2015:

In [ ]:
s_ratecountry15 = suicides[suicides['Year'] == 2015]

In [ ]:
s_ratecountry15 = s_ratecountry15[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [ ]:
s_ratecountry15['Rate'] = (s_ratecountry15['Num_Suicides']/s_ratecountry15['Population'])*1000000

In [ ]:
s_ratecountry15.sort_values('Rate', ascending = False)[0:10]

# QUESTION 6:

# Countries with the highest 5-14 years range suicide rate?

## From 1990 to 2015:

In [ ]:
bbs = suicides[suicides['Age'] == '5-14 years']

In [ ]:
bbs = bbs[['Country','Suicides/100kPop', 'Population','Num_Suicides']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)

In [ ]:
bbs.head()

In [ ]:
bbs['Rate'] = bbs['Num_Suicides']/bbs['Suicides/100kPop']

In [ ]:
bbs.sort_values('Rate', ascending = False)[0:10].head()

# POR QUE ME DA DIFERENTE???

# Countries with the highest 5-14 years range suicide rate in 2015?

In [ ]:
bbs15 = suicides.loc[(suicides['Age'] == '5-14 years') & (suicides['Year'] == 2015)]

In [ ]:
bbs15 =  bbs15[['Country','Num_Suicides','Population', 'Suicides/100kPop']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)


In [ ]:
bbs15

In [ ]:
bbs15 = bbs15.rename(columns={"Population": "Pop_5to14"})

In [ ]:
bbs15['Rate'] = bbs15['Num_Suicides']/bbs15['Pop_5to14']

In [ ]:
bbs15 = bbs15.sort_values('Rate', ascending = False)[0:10]

In [ ]:
bbs15

# POR QUE ME DA DIFERENTE???

In [ ]:
df1 = suicides_9015.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [ ]:
df1['Gen_sui/tot_sui'] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df1_f = df1[['Num_Suicides','Gen_sui/tot_sui']].round({'Gen_sui/tot_sui': 2})

In [ ]:
pop_gen = suicides_9015[['Generation','Population']].groupby('Generation').sum().sort_values('Population', ascending = False)

In [ ]:
pop_gen[''] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df2 = df.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [ ]:
df2 = df2[['Population', 'Suicides/100kPop']]

In [ ]:
df2['Population'].sum()

In [ ]:
suicides.head()

In [ ]:
suicides_lrange = suicides

In [ ]:
suicides.groupby('Year')['Country'].unique().apply(lambda x: len(x))

In [ ]:
suicides.groupby(['Country','Year']).sum().loc[('Spain',2012)]